In [1]:
import pandas as pd
ad = pd.read_csv("Advertising.csv", usecols = [1,2,3,4])
df = ad.copy()
df.head()

,TV,radio,newspaper,sales
0,230.1,37.8,69.2,22.1
1,44.5,39.3,45.1,10.4
2,17.2,45.9,69.3,9.3
3,151.5,41.3,58.5,18.5
4,180.8,10.8,58.4,12.9


In [3]:
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict

In [24]:
X = df.drop("sales", axis = 1) ## sales bağımlı değişken,bağımsız değişkenleri x'e attım
y = df["sales"]
training = df.copy() ###veri setinin tamamınıı burda tutcam
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state= 42)


In [26]:
training.shape

(200, 4)

In [7]:
X_train.shape

(160, 3)

In [9]:
X_test.shape

(40, 3)

In [16]:
y_train.shape
###(40, 3):40: Bu setin 40 örnekten (satır) oluştuğunu gösterir. 3: Bu setin her bir örneğinde 3 özellik (sütun) olduğunu gösterir.

(160,)

In [13]:
y_test.shape

(40,)

In [ ]:
###### veriyi test train diye ayırdık şimdi doğrusal regresyon modelini kurucaz.

###Statsmodels ile model oluşturma


In [49]:
###statsmodel ile
import statsmodels.api as sm
lm = sm.OLS(y_train, X_train)
model = lm.fit()
model.summary()
#model.summary().tables[1]

##scikit-learn ile model oluşturma

In [61]:
from sklearn.linear_model import LinearRegression
lm = LinearRegression()
model = lm.fit(X_train, y_train)
model.intercept_
model.coef_

array([0.04472952, 0.18919505, 0.00276111])

####tahmin

Model denklemi:
Sales = 2.97 + TV0.04 + radio0.18 + newspaper*0.02

Örneğin 30 birim TV harcaması, 10 birim radio harcamasi, 40 birimde gazete harcaması olduğunda satışların tahmini değeri ne olur?


In [69]:
from warnings import filterwarnings
filterwarnings('ignore')
yeni_veri = [[30], [10],[40]]
yeni_veri = pd.DataFrame(yeni_veri).T
model.predict(yeni_veri)

array([6.32334798])

In [78]:
import numpy as np
from sklearn.metrics import mean_squared_error
rmse = np.sqrt(mean_squared_error(y_train, model.predict(X_train))) ### train tahmin başarısına bakıcam
rmse

1.644727765644337

In [82]:
rmse = np.sqrt(mean_squared_error(y_test, model.predict(X_test))) ## test tahmin başarısına bakıcam
rmse

1.7815996615334502

In [84]:
## Model Tuning / Model Doğrulama

In [86]:
X = df.drop('sales', axis=1)
y = df["sales"]
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.20, 
                                                    random_state=144)
lm = LinearRegression() 
model = lm.fit(X_train, y_train)

In [88]:
###mesela random statei değiştirerek bu değerlerin nasıl değiştiğine bakabiliriz
np.sqrt(mean_squared_error(y_train, model.predict(X_train)))
np.sqrt(mean_squared_error(y_test, model.predict(X_test)))
model.score(X_train, y_train)

1.6748559274650712

In [98]:
## k cross validation yöntemi denicez
## predict edilince gelen değerlerin doğruluğunu test edicez
## cv burda k değeridir
cross_val_score(model, X, y, cv = 10, scoring = "r2").mean() ## bu şuan tüm veri seti için

0.8853562237979616

In [100]:
cross_val_score(model, X_train, y_train, cv = 10, scoring = "r2").mean() 

0.8733783298422942

In [102]:
np.sqrt(-cross_val_score(model, 
                X_train, 
                y_train, 
                cv = 10, 
                scoring = "neg_mean_squared_error")).mean()

1.6649345607872932

In [104]:
np.sqrt(-cross_val_score(model, 
                X_test, 
                y_test, 
                cv = 10, 
                scoring = "neg_mean_squared_error")).mean()
###neg_mean_squared_error" parametresi, ortalama karesel hata (MSE) metriğinin negatif versiyonunu kullanır. 
##Negatif olarak hesaplanmasının sebebi, cross_val_score fonksiyonunun varsayılan olarak skorları maximizasyon olarak görmesidir.
##Bu nedenle, minimize edilmesi gereken bir hata metriği olan MSE'yi negatif yaparak, 
##cross_val_score'un daha düşük hataları tercih etmesi sağlanır.
##

1.7399924960346644